# Load Env Variables

In [8]:
from decouple import config

aws_bucket_name = config('AWS_BUCKET_NAME')

# Helper Functions

In [68]:
import requests
import cv2
import numpy as np
import json

def preprocess_img(url, target_size=(224, 224)):
    try:
        res = requests.get(url)
        img = cv2.imdecode(np.frombuffer(res.content, np.uint8), -1)

        img = cv2.resize(img, target_size)
        img = img.astype(np.float32) / 255.0 # normalising in range (0, 1)

        if res.status_code == 200:
            return img    
    except Exception as e:
        print(f'Error while fetching img:{url}: {e}')

def append_to_json(json_file, nparray, word):
    try:
        with open(json_file, 'r') as file:
            data = json.load(file)
    except FileNotFoundError:
        data = {}

    data_list = data.get('data', [])

    data_list.append({
        'word': word,
        'nprray': nparray
    })

    with open(json_file, 'w') as file:
        json.dump(data, file, indent=4)

# Load Dataset

In [10]:
import json

json_file_path = '../data.json'

dataset = []

with open(json_file_path, 'r') as file:
    data = json.load(file)

for entry in data['data']:
    img_urls = []

    for url in entry['filenames']:
        s3_url = f'https://{aws_bucket_name}.s3.amazonaws.com/{url}'

        img_urls.append(s3_url)
    
    dataset.append({
        'word': entry['word'],
        'urls': img_urls
    })

In [11]:
dataset[0]

{'word': 'match',
 'urls': ['https://sign-language-recognition.s3.amazonaws.com/pictures/match/match_0.jpg',
  'https://sign-language-recognition.s3.amazonaws.com/pictures/match/match_1.jpg',
  'https://sign-language-recognition.s3.amazonaws.com/pictures/match/match_2.jpg',
  'https://sign-language-recognition.s3.amazonaws.com/pictures/match/match_3.jpg',
  'https://sign-language-recognition.s3.amazonaws.com/pictures/match/match_4.jpg',
  'https://sign-language-recognition.s3.amazonaws.com/pictures/match/match_5.jpg',
  'https://sign-language-recognition.s3.amazonaws.com/pictures/match/match_6.jpg',
  'https://sign-language-recognition.s3.amazonaws.com/pictures/match/match_7.jpg',
  'https://sign-language-recognition.s3.amazonaws.com/pictures/match/match_8.jpg',
  'https://sign-language-recognition.s3.amazonaws.com/pictures/match/match_9.jpg',
  'https://sign-language-recognition.s3.amazonaws.com/pictures/match/match_10.jpg',
  'https://sign-language-recognition.s3.amazonaws.com/pictur

In [12]:
# Range of pixel values

import cv2
import urllib
import numpy as np

img_url = dataset[3]['urls'][3].replace(' ', '+')

req = urllib.request.urlopen(img_url)
arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
img = cv2.imdecode(arr, -1)

min_val = img.min()
max_val = img.max()

print("Minimum pixel value:", min_val)
print("Maximum pixel value:", max_val)

Minimum pixel value: 0
Maximum pixel value: 226


In [28]:
# for entry in dataset[0]:

images = []

for entry in dataset[0:2]:
    for url in entry['urls']:
        img = preprocess_img(url)

        images.append({
            'word': dataset[0]['word'],
            'nparray': img
        })

print(images)

[{'word': 'match', 'nparray': array([[[0.05882353, 0.04705882, 0.03137255],
        [0.0627451 , 0.05098039, 0.03529412],
        [0.06666667, 0.05490196, 0.03921569],
        ...,
        [0.9490196 , 0.9607843 , 0.9254902 ],
        [0.9529412 , 0.9647059 , 0.9254902 ],
        [0.9529412 , 0.96862745, 0.9254902 ]],

       [[0.05882353, 0.04705882, 0.03137255],
        [0.0627451 , 0.05098039, 0.03137255],
        [0.06666667, 0.05490196, 0.03921569],
        ...,
        [0.9490196 , 0.9607843 , 0.9254902 ],
        [0.9490196 , 0.9647059 , 0.9254902 ],
        [0.9529412 , 0.96862745, 0.9254902 ]],

       [[0.05882353, 0.04705882, 0.03137255],
        [0.0627451 , 0.05098039, 0.03529412],
        [0.06666667, 0.05490196, 0.03921569],
        ...,
        [0.9490196 , 0.9607843 , 0.9254902 ],
        [0.9490196 , 0.9647059 , 0.9254902 ],
        [0.9529412 , 0.96862745, 0.9254902 ]],

       ...,

       [[0.05490196, 0.03921569, 0.03529412],
        [0.05490196, 0.03921569, 0.035

In [75]:
import numpy as np

images = []

for entry in dataset:
    urls = entry['urls']
    word = entry['word']

    for url in urls:
        img = preprocess_img(url)

    images.append({
        'word': word,
        'nparray': img
    })

    np.save(f'../images/{word}.npy', img)

print(images)

[{'word': 'match', 'nparray': array([[[0.0627451 , 0.05098039, 0.03529412],
        [0.0627451 , 0.05098039, 0.03529412],
        [0.05882353, 0.04705882, 0.03137255],
        ...,
        [0.07450981, 0.05882353, 0.05490196],
        [0.07843138, 0.0627451 , 0.05882353],
        [0.07843138, 0.0627451 , 0.05882353]],

       [[0.0627451 , 0.05098039, 0.03529412],
        [0.0627451 , 0.05098039, 0.03529412],
        [0.05882353, 0.04705882, 0.03137255],
        ...,
        [0.07450981, 0.05882353, 0.05490196],
        [0.07843138, 0.0627451 , 0.05882353],
        [0.07843138, 0.0627451 , 0.05882353]],

       [[0.0627451 , 0.05098039, 0.03529412],
        [0.0627451 , 0.05098039, 0.03529412],
        [0.05882353, 0.04705882, 0.03137255],
        ...,
        [0.07450981, 0.05882353, 0.05490196],
        [0.07843138, 0.0627451 , 0.05882353],
        [0.07843138, 0.0627451 , 0.05882353]],

       ...,

       [[0.8980392 , 0.9019608 , 0.8627451 ],
        [0.89411765, 0.8980392 , 0.858

In [74]:
import tensorflow as tf
import numpy as np

words = [entry['word'] for entry in dataset]
vocab = set(words)

word_to_index = {word: index for index, word in enumerate(vocab)}

labels_list = []

for word in vocab:
    labels = [word_to_index[word] for word in words]  # Use -1 for words not matching the current one
    labels_tensor = tf.constant(labels).numpy()

    labels_list.append({
        'word': word,
        'nparray': labels_tensor
    })

    np.save(f'../labels/{word}.npy', labels_tensor)
    
print(labels_list)

[{'word': 'flat tire', 'nparray': array([208, 514, 694, ..., 386, 423, 495], dtype=int32)}, {'word': 'not', 'nparray': array([208, 514, 694, ..., 386, 423, 495], dtype=int32)}, {'word': 'bird', 'nparray': array([208, 514, 694, ..., 386, 423, 495], dtype=int32)}, {'word': 'mushroom', 'nparray': array([208, 514, 694, ..., 386, 423, 495], dtype=int32)}, {'word': 'elevator', 'nparray': array([208, 514, 694, ..., 386, 423, 495], dtype=int32)}, {'word': 'always', 'nparray': array([208, 514, 694, ..., 386, 423, 495], dtype=int32)}, {'word': 'cent', 'nparray': array([208, 514, 694, ..., 386, 423, 495], dtype=int32)}, {'word': 'stay', 'nparray': array([208, 514, 694, ..., 386, 423, 495], dtype=int32)}, {'word': 'forget', 'nparray': array([208, 514, 694, ..., 386, 423, 495], dtype=int32)}, {'word': 'sophomore', 'nparray': array([208, 514, 694, ..., 386, 423, 495], dtype=int32)}, {'word': 'jealous', 'nparray': array([208, 514, 694, ..., 386, 423, 495], dtype=int32)}, {'word': 'secretary', 'nparra

In [47]:
print(labels_list[0])
print(f'Length of labels: {len(labels_list)}') # should be 791

{'word': 'flat tire', 'nparray': array([-1, -1, -1, ..., -1, -1, -1], dtype=int32)}
Length of labels: 791
